In [ ]:
import numpy as np
import pandas as pd
import os
import re
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import csv
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA, NMF
from sklearn.preprocessing import StandardScaler
from sklearn import svm

# Paramétrage de l'affichage des dataframes
pd.set_option('display.max_columns', 12)
pd.set_option('display.max_rows', None)

In [ ]:
# Détermination du temps de vieillissement à partir du nom de fichier
def age(parameters):
    compound_mapping = {
        '240208': 349,
        '240308': 378,
        '240405': 406,
        '240503': 434,
        '240607': 462,
        'frais': 0
    }
    
    for key, value in compound_mapping.items():
        if key in parameters:
            return value
    return 0

In [ ]:
# Indiquer l'emplacement des données
directory =

dataframes = []
tapes = []
triplicats = []
filenames = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        
        try:
            df = pd.read_csv(file_path, skiprows=12, delimiter='\t', names=['Ret.Time', 'Absolute Intensity', 'Relative Intensity'], 
                             dtype={'Ret.Time': float, 'Absolute Intensity': int, 'Relative Intensity': float}, 
                             index_col='Ret.Time', decimal=',', encoding='utf-8')
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, skiprows=12, delimiter='\t', names=['Ret.Time', 'Absolute Intensity', 'Relative Intensity'], 
                                 dtype={'Ret.Time': float, 'Absolute Intensity': int, 'Relative Intensity': float}, 
                                 index_col='Ret.Time', decimal=',', encoding='latin1')
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, skiprows=12, delimiter='\t', names=['Ret.Time', 'Absolute Intensity', 'Relative Intensity'], 
                                 dtype={'Ret.Time': float, 'Absolute Intensity': int, 'Relative Intensity': float}, 
                                 index_col='Ret.Time', decimal=',', encoding='cp1252')

        parameters = re.split(r'[-._ ]', filename)
        tapes.append(parameters[0])
        triplicats.append(parameters[-2])
        filenames.append(filename)
        dataframes.append(df)

data = pd.concat(dataframes, axis=1)
data = data.loc[:, ~data.columns.str.contains('Relative Intensity')]
data = data.T.reset_index(drop=True).rename_axis('Index')

data['tape'] = tapes
data['triplicat'] = triplicats
data['file name'] = filenames

cols = list(data.columns)
cols = ['tape'] + [col for col in cols if col != 'tape']
data = data[cols]

data.sort_values(by=['tape', 'triplicat'], inplace=True)
data.reset_index(drop=True, inplace=True)

data

In [ ]:
# Les données n'étant pas rigoureusement collectées touttes les 0.001 min,
# seules les colonnes finissant en 0.000 ou 0.005 sont conservées,
# afin de permettre de décaler les chromato pour les supperposer
selected_columns = [col for col in data.columns[1:-2] if str(col).endswith(('5', '0'))]
data_selected = data.loc[:, ['tape'] + list(selected_columns) + ['triplicat', 'file name']]

data_selected

In [ ]:
# Défini les décallages pour chaque chromatogramme
shift_values = {
    'raa12': {'1': -0.02, '2': -0.015, '3': -0.020},
    'raa13': {'2': -0.00, '3': -0.05},
    'raa14': {'1': -0.03, '2': -0.03, '3': -0.02},
    'raa15': {'2': -0.00, '3': +0.005},
    'raa16': {'2': -0.00, '3': -0.02}
}

# Converti le décallage temporel en décallage de colonnes
def convert_shifts_to_columns(shift_values, step_size):
    column_shifts = {}
    for tape, triplicat_shifts in shift_values.items():
        column_shifts[tape] = {triplicat: int(shift / step_size) for triplicat, shift in triplicat_shifts.items()}
    return column_shifts
step_size = 0.002
column_shifts = convert_shifts_to_columns(shift_values, step_size)

# Décale les données dans le dataframe
def shift_columns(data, column_shifts):
    non_numeric_cols = ['tape', 'triplicat', 'file name']
    numeric_data = data.drop(columns=non_numeric_cols)
    shifted_data = numeric_data.transpose()  # Transpose the numeric part of the DataFrame
    for idx, row in data.iterrows():
        tape = row['tape']
        triplicat = row['triplicat']
        if tape in column_shifts and triplicat in column_shifts[tape]:
            shift = column_shifts[tape][triplicat]
            shifted_data[idx] = shifted_data[idx].shift(periods=-shift)
    
    shifted_numeric_data = shifted_data.transpose()  # Transpose it back
    # Combine the 'tape' column with the shifted numeric data
    final_data = pd.concat([data[['tape']], shifted_numeric_data], axis=1)
    # Add 'age' and 'file name' columns back in the end
    final_data = pd.concat([final_data, data[['triplicat', 'file name']]], axis=1)
    return final_data

In [ ]:
# Applique le décalage
data_shifted = shift_columns(data, column_shifts)

data_raa12_shifted = data_shifted[data_shifted["tape"]=="raa12"]
data_raa13_shifted = data_shifted[data_shifted["tape"]=="raa13"]
data_raa14_shifted = data_shifted[data_shifted["tape"]=="raa14"]
data_raa15_shifted = data_shifted[data_shifted["tape"]=="raa15"]
data_raa16_shifted = data_shifted[data_shifted["tape"]=="raa16"]

In [ ]:
# Représente le chromatogramme, avec possibilité de réduire la plage temporelle,
# et d'afficher les pics identifiés.
def plot_TIC(data_plot_spectra, start_time=None, end_time=None, found_tr=None):
    Tr = data_plot_spectra.columns[1:-3].astype(float)
    data_to_plot = data_plot_spectra.values[:, 1:-3]

    if start_time is not None:
        start_idx = np.searchsorted(Tr, start_time, side='left')
    else:
        start_idx = 0

    if end_time is not None:
        end_idx = np.searchsorted(Tr, end_time, side='right')
    else:
        end_idx = len(Tr)

    Tr_filtered = Tr[start_idx:end_idx]
    data_filtered = data_to_plot[:, start_idx:end_idx]

    plt.figure(figsize=(12, 4))
    cmap = cm.coolwarm(np.linspace(0, 0.8, len(data_filtered)))

    for row in range(len(data_filtered)):
        Int = data_filtered[row]
        color = cmap[row]
        triplicat = data_plot_spectra.values[row, data_plot_spectra.columns.get_loc('triplicat')]
        if triplicat == 0:
            color = 'black'
            label = 'Triplicat: 0'
        else:
            label = f'Triplicat: {triplicat}'
        plt.plot(Tr_filtered, Int, color=color, linestyle='-', linewidth=0.5, alpha=1, label=label)
    
    if found_tr is not None:
            plt.axvline(x=found_tr, color='black', linestyle='--', linewidth=1, label=f'Found TR: {found_tr}')
    
    plt.xlabel('Temps de rétention (min)')
    plt.ylabel('Intensité')
    plt.legend()
    plt.show()

# Représente les chromatrogrammes avec un décalage vertical
def plot_with_vshift(data, vshift, start_time=None, end_time=None, found_tr=None):
    data2 = data.copy()
    numeric_columns = data.columns[1:-3]  # Assuming columns from the second to the third last
    i = 1
    for idx, row in data.iterrows():
        data2.loc[idx, numeric_columns] = row[numeric_columns].astype(float) + i*vshift
        i += 1
    return data2

In [ ]:
datadata_raa12_vshifted = plot_with_vshift(data_raa12_shifted, 1000000)
datadata_raa13_vshifted = plot_with_vshift(data_raa13_shifted, 1000000)
datadata_raa14_vshifted = plot_with_vshift(data_raa14_shifted, 1000000)
datadata_raa15_vshifted = plot_with_vshift(data_raa15_shifted, 1000000)
datadata_raa16_vshifted = plot_with_vshift(data_raa16_shifted, 1000000)

plot_TIC(datadata_raa12_vshifted, 1.11, 38)
plot_TIC(datadata_raa13_vshifted, 1.11, 38)
plot_TIC(datadata_raa14_vshifted, 1.11, 38)
plot_TIC(datadata_raa15_vshifted, 1.11, 38)
plot_TIC(datadata_raa16_vshifted, 1.11, 38)

## Data processing

In [ ]:
# Définition des fonctions utilisées pour le calcul des intensités
# Ajustement des paramètres de recherche des pics manuellement
def closest_inferior_value(value, value_list):
    closest = None
    for v in value_list:
        if v < value:
            if closest is None or value - v < value - closest:
                closest = v
    return closest

# Ajuste le temps de rétention
def find_max_intensity(df, row, retention_time, window=0.1):
    
    if df.loc[row, 'tape'] == 'raa12':
        if retention_time > 4:
            if retention_time < 4.5:
                window = 0.2
        if retention_time > 16.6:
            if retention_time < 16.7:
                window = 0.03
    
    if df.loc[row, 'tape'] == 'raa13':
        if retention_time > 4.3:
            if retention_time < 4.6:
                window = 0.2
        if retention_time > 7:
            if retention_time < 7.5:
                window = 0.2
        if retention_time > 19:
            if retention_time < 19.5:
                window = 0.2
        if df.loc[row, 'triplicat'] == '3':
            if retention_time > 2.5:
                if retention_time < 3.7:
                    window = 0.05
            if retention_time > 3.7:
                if retention_time < 3.8:
                    window = 0.02
            if retention_time > 3.8:
                if retention_time < 4:
                    window = 0.05
            if retention_time > 10.31:
                if retention_time < 10.5:
                    window = 0.04
            if retention_time > 11:
                if retention_time < 11.3:
                    window = 0.05
            if retention_time > 11.4:
                if retention_time < 12.5:
                    window = 0.2
            if retention_time > 13.5:
                if retention_time < 14.1:
                    window = 0.2
            if retention_time > 14.5:
                if retention_time < 14.6:
                    window = 0.05
            if retention_time > 14.9:
                if retention_time < 15:
                    window = 0.1
            if retention_time > 15:
                if retention_time < 15.9:
                    window = 0.1
            if retention_time > 15.9:
                if retention_time < 18.25:
                    window = 0.1
            if retention_time > 18.4:
                if retention_time < 19:
                    window = 0.1
            if retention_time > 20:
                if retention_time < 30:
                    window = 0.2
            
    if df.loc[row, 'tape'] == 'raa14':
        if retention_time > 1:
            if retention_time < 2:
                window = 0.1
        if retention_time > 2:
            if retention_time < 2.5:
                window = 0.025
        if retention_time > 6:
            if retention_time < 7.5:
                window = 0.1
        if retention_time > 7.5:
            if retention_time < 7.9:
                window = 0.05
        if retention_time > 7.9:
            if retention_time < 8:
                window = 0.01
        if retention_time > 8.2:
            if retention_time < 10:
                window = 0.1
        if retention_time > 11.8:
            if retention_time < 11.9:
                window = 0.1
    
    if df.loc[row, 'tape'] == 'raa15':
        if retention_time > 1:
            if retention_time < 2:
                window = 0.1
        if retention_time > 8:
            if retention_time < 13.5:
                window = 0.1
        if retention_time > 19.5:
            if retention_time < 20:
                window = 0.1
        if retention_time > 32:
            if retention_time < 35:
                window = 0.1
    
    if df.loc[row, 'tape'] == 'raa16':
        if retention_time > 3.7:
            if retention_time < 3.8:
                window = 0.05
                
    retention_times = df.columns[10:-100].astype(float)
    start_time = closest_inferior_value(retention_time - window, retention_times)
    end_time = closest_inferior_value(retention_time + window, retention_times)
    max_intensity = df.loc[row, start_time:end_time].max()
    max_retention_time = df.loc[row, start_time:end_time].idxmax()

    return max_intensity, max_retention_time


def relative_intensity(df, row, retention_time_1, retention_time_2):    
    I_peak = find_max_intensity(df, row, retention_time_1)[0]
    ref_peak = find_max_intensity(df, row, retention_time_2)[0]
    return I_peak / ref_peak

In [ ]:
# Recherche tous les pics au dessus d'une valeur seuil
def find_peak_rt(df, row, start, end, threshold):
    Tr = df.columns[1:-3].astype(float)
    intensity_values = df.iloc[row, 1:-3].values
    derivate = np.gradient(intensity_values)
    
    # Filter the retention times and intensities based on start and end
    start_idx = np.searchsorted(Tr, start, side='left')
    end_idx = np.searchsorted(Tr, end, side='right')
    Tr_filtered = Tr[start_idx:end_idx]
    intensity_filtered = intensity_values[start_idx:end_idx]
    derivate_filtered = derivate[start_idx:end_idx]

    # Step 1: Find peaks using find_peaks
    peaks, _ = find_peaks(intensity_filtered, height=threshold)
    
    # Step 2: Find the retention time where the derivative crosses 0 closest to the found peaks
    zero_crossings = np.where(np.diff(np.sign(derivate_filtered)))[0]
    peak_rts = []
    peak_rts2 = []
    
    for peak_idx in peaks:
        peak_ret_time = Tr_filtered[peak_idx]
        closest_zero_crossing_idx = np.argmin(np.abs(Tr_filtered[zero_crossings] - peak_ret_time))
        closest_zero_crossing = zero_crossings[closest_zero_crossing_idx]
        peak_rts.append(Tr_filtered[closest_zero_crossing])
        peak_rts2.append(Tr_filtered[peak_idx])
    
    return peak_rts

In [ ]:
# Détermine l'étalement du pic
# Ajustement manuel
def find_peak_range(df, row, start, end, peak):
    if df.loc[row, 'tape'] == 'raa12':
        if peak > 7:
            if peak < 7.2:
                return 7.025, 7.17
        if peak > 7.2:
            if peak < 7.4:
                return 7.28, 7.43
        if peak > 16:
            if peak < 16.5:
                return 16.23, 16.33
        if peak > 21:
            if peak < 22:
                return 21.74, 21.83
        
    if df.loc[row, 'tape'] == 'raa13':
        if peak > 1:
            if peak < 2.5:
                return 1.8, 2.3  
        if peak > 6.5:
            if peak < 7:
                return 6.815, 7
        if peak > 7:
            if peak < 7.5:
                return 7.41, 7.6
        if peak > 7.5:
            if peak < 8:
                return 7.9, 8.05
        if df.loc[row, 'triplicat'] == '3':
            if peak > 15.9:
                if peak < 16:
                    return 15.92, 16
            if peak > 26.7:
                if peak < 26.82:
                    return 26.73, 26.84
            if peak > 26.81:
                if peak < 26.92:
                    return 26.84, 26.95
        if peak > 26.7:
            if peak < 26.81:
                return 26.7, 26.81
        if peak > 26.81:
            if peak < 26.92:
                return 26.81, 26.92

    if df.loc[row, 'tape'] == 'raa14':
        if df.loc[row, 'triplicat'] == '3':
            if peak > 2.1:
                if peak < 2.22:
                    return 2.14, 2.21
            if df.loc[row, 'triplicat'] == '1':
                if peak > 7.94:
                    if peak < 8.05:
                        return 7.94, 8.05
            if df.loc[row, 'triplicat'] == '2':
                if peak > 7.94:
                    if peak < 8.05:
                        return 7.94, 8.05
        
    if df.loc[row, 'tape'] == 'raa15':
        if peak > 13.5:
            if peak < 14:
                return 13.45, 14.05
        if peak > 32.9:
            if peak < 33.05:
                return 32.9, 33.05
    
    if df.loc[row, 'tape'] == 'raa16':
        if peak > 4:
            if peak < 4.1:
                return 4.01, 4.08
        if peak > 4.1:
            if peak < 4.2:
                return 4.08, 4.19
        if peak > 10.2:
            if peak < 10.255:
                return 10.2, 10.255

    Tr = df.columns[1:-3].astype(float)
    intensity_values = df.iloc[row, 1:-3].values
    derivate = np.gradient(intensity_values)
    
    new_peak = find_max_intensity(df, row, peak, window=0.02)[1]
    peak = new_peak
                                                              
    start_idx = np.searchsorted(Tr, start, side='left')
    end_idx = np.searchsorted(Tr, end, side='right')
    Tr_filtered = Tr[start_idx:end_idx]
    intensity_filtered = intensity_values[start_idx:end_idx]
    derivate_filtered = derivate[start_idx:end_idx]
    
    peak_index = np.searchsorted(Tr_filtered, peak)
    start = None
    i = 2
    while start == None:
        if derivate_filtered[peak_index-i] <= 0:
            start = peak_index-i
        i += 1
    end = None
    i = 2
    while end == None:
        if derivate_filtered[peak_index+i] >= 0:
            end = peak_index+i
        i += 1
    start_tr = Tr_filtered[start]
    end_tr = Tr_filtered[end]

    return start_tr, end_tr

In [ ]:
# Illustration de comment sont trouvés les maximums et bases des pics
def plot_TIC_with_derivate(data_plot_spectra, row=None, start_time=None, end_time=None, peaks_info=None):
    # Extract retention times and intensities
    Tr = data_plot_spectra.columns[1:-3].astype(float)
    if row is not None:
        intensity_values = data_plot_spectra.iloc[row, 1:-3].values
        data_to_plot = np.array([intensity_values])
        derivate = np.gradient(intensity_values)
    else:
        data_to_plot = data_plot_spectra.values[:, 1:-3]

    # Filter columns based on the specified start and end retention times
    if start_time is not None:
        start_idx = np.searchsorted(Tr, start_time, side='left')
    else:
        start_idx = 0

    if end_time is not None:
        end_idx = np.searchsorted(Tr, end_time, side='right')
    else:
        end_idx = len(Tr)

    Tr_filtered = Tr[start_idx:end_idx]
    data_filtered = data_to_plot[:, start_idx:end_idx]
    derivate_filtered = derivate[start_idx:end_idx]
    fig, ax1 = plt.subplots(figsize=(12, 6))
    
    cmap = cm.viridis(np.linspace(0, 0.8, len(data_filtered)))

    for row in range(len(data_filtered)):
        Int = data_filtered[row]
        color = cmap[row]
        triplicat = data_plot_spectra.values[row, data_plot_spectra.columns.get_loc('triplicat')]
        if triplicat == 0:
            color = 'red'
            label = 'Triplicat: 0'
        else:
            label = f'Triplicat: {triplicat}'
        ax1.plot(Tr_filtered, Int, color=color, linestyle='-', linewidth=0.5, alpha=1, label=label)
    
    if peaks_info is not None:
        for peak_info in peaks_info:
            peak_rt, start_rt, end_rt = peak_info

            ax1.axvline(x=peak_rt, color='black', linestyle='--', linewidth=1, label=f'Found TR: {peak_rt}')
            ax1.axvline(x=start_rt, color='red', linestyle='--', linewidth=1, alpha=0.5)
            ax1.axvline(x=end_rt, color='red', linestyle='--', linewidth=1, alpha=0.5)

    ax1.set_xlabel('Temps de rétention (min)')
    ax1.set_ylabel('Intensité')
    ax1.legend()
    
    ax2 = ax1.twinx()

    ax2.plot(Tr_filtered, derivate_filtered, color='blue', linestyle='-', linewidth=0.5, alpha=1)
    ax2.set_ylabel('First Derivative', color='blue')
    
    plt.show()

peaks_info = []
peak_rts = find_peak_rt(data, 0, 12, 14.5, 250000)
for peak in peak_rts:
    peak_info = []
    peak_info.append(peak)
    peak_info.append(find_peak_range(data, 0, 12, 14.5, peak)[0])
    peak_info.append(find_peak_range(data, 0, 12, 14.5, peak)[1])
    peaks_info.append(peak_info)

print(peaks_info)

plot_TIC_with_derivate(data_shifted[:], row=0, start_time=12, end_time=14.5, peaks_info=peaks_info)

In [ ]:
# Calcul de l'air d'un pic
def peak_area(df, row, start, end):
    Tr = df.columns[1:-3].astype(float)
    intensity_values_a = df.iloc[row, 1:-3].values
    
    start_idx = np.searchsorted(Tr, start, side='left')
    end_idx = np.searchsorted(Tr, end, side='right')
    Tr_filtered_a = Tr[start_idx:end_idx]
    intensity_filtered_a = intensity_values_a[start_idx:end_idx]
    
    baseline_start = intensity_filtered_a[0]
    baseline_end = intensity_filtered_a[-1]
    baseline_min = min(baseline_start, baseline_end)
    baseline = np.linspace(baseline_min, baseline_min, len(Tr_filtered_a))
    
    area = np.trapz(intensity_filtered_a, Tr_filtered_a)*60
    
    return area, Tr_filtered_a, intensity_filtered_a, baseline

## Plot

In [ ]:
# Définition des temps de rétention approximatifs, des noms de composés et de leur catégorie

# RAA12
retention_times_raa12 = [4.195, 7.09, 7.34, 7.95, 8.14, 9.47, 9.52, 10.71, 11.06, 11.39,
                         
                         13.56, 13.92, 14.15, 15.62, 15.85, 16.27, 16.69, 16.75, 16.79, 16.92, 
                         
                         17, 17.98, 18.06, 18.13, 18.2, 19.28, 19.47, 21.78, 22.87, 26.89, 
                         
                         27.22, 27.42, 28.02]

compounds_raa12 = ['toluène', 'éthylbenzène', 'm-xylène', 'styrène', 'C9H14', 'C10H16', '5-éthényl-1,5-diméthylcyclohexène', 
                   '2,5-diméthyl-3-méthylidènehepta-1,5-diène', 'D-limonène', '1H-indène', 
                   
                   '1-méthyl-1H-indène', 'naphtalène', 'isothiocyanatobenzène', '2-méthylnaphtalène', '1-méthylnaphtalène', 
                   '2-tert-butyl-6-méthylphénol', '?', 'C14H28', '1,2-dihydroacénaphthylène', '?',
                   
                   '?', '?', '?', '?', '?', '?', '9H-fluorène', 'anthracène', 'C15H24', '?', 
                   
                   '?', '2,2\'-méthylènebis(6-tert-butyl-4-méthylphénol) ', '?']

compound_types_raa12 = ['Support ou masse adhésive', 'Support', 'Support ou masse adhésive', 'Support', 'Alcène', 'Alcène', 'Masse adhésive', 
                        'Masse adhésive', 'Masse adhésive', 'Support', 
                        
                        'Support', 'Support', 'Additif', 'Support', 'Support', 'Additif', 'Alcène', 'Alcène', 'Support', 
                        'Alcène', 
                        
                        'Alcène', 'Alcène', 'Alcène', 'Indéterminé', 'Mélange', 'Alcène', 'Support', 'Support', 'Alcène', 'Indéterminé', 
                        
                        'Alcane', 'Additif', 'Alcane']

compound_types_plot_raa12 = ['Support', 'Support ou masse adhésive', 'Masse adhésive', 'Additif', 'Alcène', 'Alcane', 'Mélange', 'Indéterminé']


# RAA13
retention_times_raa13 = [2.15, 2.9, 3.02, 3.26, 3.37, 3.66, 3.73, 3.86, 4.06, 4.42,
                         
                        4.83, 6.85, 7.41, 7.93, 10.3, 10.38, 10.91, 11.08, 11.13, 11.2,
                         
                        11.33, 11.85, 12.18, 12.51, 13.61, 13.98, 14.42, 14.59, 14.65, 14.7,
                         
                        14.79, 14.87, 14.93, 15.39, 15.52, 15.67, 15.85, 15.92, 16.15, 16.51,
                         
                        18.22, 18.29, 18.83, 19.35, 20.09, 26.4, 26.76, 26.87, 27.21]

compounds_raa13 = ['acide acétique', '3,5-diméthylhex-1-ène', 'méthacrylate de méthyle', 'anhydride acétique', '4-méthylheptan-1-ol',
                  '?', '?', '?', '5-méthylhept-1-ène', 'acétate de 2-hydroxyéthyle',
                  
                  'oct-1-ène', '?', 'acétate de 2-oxopropyle', 'styrène', '6-méthylheptan-1-ol', 'acétate de furfuryle ?',
                  '3-éthyl-4-méthyl-1-pentanol', '?', '3-méthylheptan-1-ol', '4-méthylheptan-1-ol',
                  
                  '5-méthylheptan-1-ol', 'octan-1-ol', '?', '?', 'benzoate d\'éthyle', 'prop-2-énoate d\'octyle', '?', '?', '?',
                  'Ethylene, diacrylate',
                  
                  'diacrylate d\'éthylène', '?', '?', '?', '?', 'acétate de (5-formylfuran-2-yl)méthyle', 'anhydride phtalique', '?', '?', '?',
                   
                  '?', '2,6-di-tert-butylphénol', '?', 'phtalate de diéthyle (DEP)', '?', '?', '?', '?', '?']

compound_types_raa13 = ['Support', 'Alcène', 'Masse adhésive', 'Support', 'Alcool', 'Alcène', 'Alcène', 'Alcène', 'Alcène', 'Support',
                      
                      
                      'Alcène', 'Support', 'Alcène', 'Support', 'Ester', 'Aromatique', 'Alcool', 'Terpène', 'Alcool',
                      'Alcool',
                      
                      'Alcool', 'Alcool', 'Alcool', 'Alcool', 'Ester', 'Ester', 'Ester', 'Ester', 'Ester', 'Ester',
                      
                      'Ester', 'Ester', 'Alcool', 'Ester', 'Indéterminé', 'Ester', 'Additif', 'Support', 'Support', 'Support',
                      
                      'Support', 'Additif', 'Ester', 'Additif', 'Support', 'Indéterminé', 'Indéterminé', 'Indéterminé', 'Indéterminé']

compound_types_plot_raa13 = ['Support', 'Masse adhésive', 'Additif', 'Alcool', 'Alcène' ,'Ester', 'Aromatique', 
                             'Terpène', 'Indéterminé']

# RAA14
retention_times_raa14 = [1.87, 2.2, 2.24, 2.33, 6.46, 7.89, 7.95, 8.42, 9.52, 
                         
                         10.25, 10.39, 10.71, 11.04, 11.38, 11.86, 11.97, 12.71, 13.12, 15.46,
                         
                         15.58, 15.7, 16.08, 16.44]

compounds_raa14 = ['2-méthylpent-1-ène', '?', '?', '?', '2,4-diméthylhept-1-ène', '2,4,6-triméthylhept-1-ène', 'styrène',
                   '2,4,6-triméthylhepta-1,6-diène', '5-éthényl-1,5-diméthylcyclohexène',
                   
                   '4,6-diméthylnon-2-ène', '?', '2,5-diméthyl-3-méthylidènehepta-1,5-diène', 'D-limonène', '?',
                   '2,4,6-triméthylnon-1-ène (méso)', '2,4,6-triméthylnon-1-ène (racémique)', '2,4,6,8-tétraméthylnon-1-ène',
                   '2,4,6,8-tétraméthylnona-1,8-diène', '2,4,6,8-tétraméthylundéc-1-ène (isotactique)',
                   
                   '2,4,6,8-tétraméthylundéc-1-ène (hétérotactique)', '2,4,6,8-tétraméthylundéc-1-ène (syndiotactique)', '2,4,6,8,10-pentaméthylundéc-1-ène (isotactique)', 
                   '2,4,6,8,10-pentaméthylundécadi-1,10-ène (isotactique)']

compound_types_raa14 = ['Support', 'Support', 'Support', 'Support', 'Support', 'Support', 'Masse adhésive', 'Support', 'Masse adhésive',
                        
                        'Support', 'Alcène', 'Masse adhésive', 'Masse adhésive', 'Aromatique', 'Support', 'Support', 'Support', 'Support', 'Support',
                        
                        'Support', 'Support', 'Support', 'Support']

compound_types_plot_raa14 = ['Support', 'Masse adhésive', 'Alcène', 'Aromatique']


# RAA15
retention_times_raa15 = [1.98, 2.43, 3.03, 8.12, 10.07, 13.05, 13.9, 16.79, 16.87, 17.08, 
                         
                         17.27, 19.05, 19.66, 20.1, 20.61, 21.03, 22.55, 22.75, 22.8, 25.35, 
                         
                         26.81, 26.88, 27.54, 28.07, 29.1, 32.99]

compounds_raa15 = ['butanal', 'butan-1-ol / benzène', 'méthacrylate de méthyle ', 'acrylate de butyle', 
                   'méthacrylate de butyle', 'benzoate de vinyle', 'acide benzoïque',
                   'biphényle', 'acide 4-vinylbenzoïque', 'benzoate de 2-chloroéthyle', 
                   
                   'benzoate de 2-hydroxyéthyle', 'téréphtalate de divinyle', 'acide 4-(vinyloxycarbonyl)benzoïque', '?', '?', '?', '?', '?', '?', 
                   'dibenzoate d\'éthylène',
                   
                   '?', 'téréphtalate de di-2-chloroéthyle', '?', '?', 'téréphtalate de 2-(benzoyloxy)éthyle et de vinyle', '?']

compound_types_raa15 = ['Aldéhyde', 'Mélange', 'Masse adhésive', 'Masse adhésive', 'Masse adhésive', 'Support', 'Support', 'Support', 'Support', 'Support', 
                        
                        'Support', 'Support', 'Support', 'Diester', 'Indéterminé', 'Mélange', 'Dérivé benzoïque', 'Indéterminé', 'Indéterminé', 'Dérivé benzoïque', 
                        
                        'Mélange', 'Dérivé benzoïque chloré', 'Indéterminé', 'Indéterminé', 'Dérivé benzoïque', 'Indéterminé']

compound_types_plot_raa15 = ['Support', 'Masse adhésive', 'Dérivé benzoïque', 'Dérivé benzoïque chloré', 'Diester', 'Aldéhyde', 'Mélange', 'Indéterminé']


# RAA16
retention_times_raa16 = [2.9, 3.03, 3.38, 3.66, 3.75, 3.88, 4.06, 4.12, 6.44, 7.89, 
                         
                         7.94, 8.42, 10.25, 10.3, 10.92, 11.08, 11.14, 11.21, 11.34, 11.88, 
                         
                         11.96, 12.19, 12.71, 13.11, 13.99, 14.66, 14.71, 14.8, 14.88, 14.95, 
                         
                         15.46, 15.58, 15.7, 16.08, 16.44]

compounds_raa16 = ['3,5-diméthylhex-1-ène', 'méthacrylate de méthyle', '?', '?', 'C8H16', 'C8H16', 'C8H16', '?',
                   '2,4-diméthylhept-1-ène', '2,4,6-triméthylhept-1-ène', 
                   
                   'styrène', '2,4,6-triméthylhepta-1,6-diène', '4,6-diméthylnon-2-ène',
                   '6-méthylheptan-1-ol', '3-éthyl-4-méthylpentan-1-ol', '?', 'octan-1-ol', '4-méthylheptan-1-ol',
                   '5-méthylheptan-1-ol', '2,4,6-triméthylnon-1-ène (méso)', 
                   
                   '2,4,6-triméthylnon-1-ène (racémique)', '?', '2,4,6,8-tétraméthylnon-1-ène', '2,4,6,8-tétraméthylnona-1,8-diène', 
                   '?', '?', '?', '?', '?', '?', 
                   
                   '2,4,6,8-tétraméthylundéc-1-ène (isotactique)', '2,4,6,8-tétraméthylundéc-1-ène (hétérotactique)',
                   '2,4,6,8-tétraméthylundéc-1-ène (syndiotactique)', '2,4,6,8,10-pentaméthylundéc-1-ène (isotactique)', 
                   '2,4,6,8,10-pentaméthylundécadi-1,10-ène (isotactique)']

compound_types_raa16 = ['Alcène', 'Masse adhésive', 'Alcène', 'Mélange', 'Alcène', 'Alcène', 'Mélange', 'Mélange', 'Support', 'Support', 
                        
                        'Aromatique', 'Support', 'Support', 'Alcool', 'Alcool', 'Alcène', 'Alcool', 'Alcool', 'Alcool', 'Support', 
                        
                        'Support', 'Alcool', 'Support', 'Support', 'Ester', 'Ester', 'Ester', 'Ester', 'Indéterminé', 'Indéterminé', 
                        
                        'Support', 'Support', 'Support', 'Support', 'Support']

compound_types_plot_raa16 = ['Support', 'Masse adhésive', 'Alcène', 'Alcool', 'Ester', 'Aromatique', 'Mélange', 'Indéterminé']

In [ ]:
# Calcul des intensités relatives pour le graph
def initialise_plot_bars_data():
    print(raa_type)
    if raa_type == 'raa12':
        retention_times = retention_times_raa12
        compounds = compounds_raa12
        compound_types = compound_types_raa12
        compound_types_plot = compound_types_plot_raa12
        data_raa = data[data["tape"]=="raa12"]
    if raa_type == 'raa13':
        retention_times = retention_times_raa13
        compounds = compounds_raa13
        compound_types = compound_types_raa13
        compound_types_plot = compound_types_plot_raa13
        data_raa = data[data["tape"]=="raa13"]
    if raa_type == 'raa14':
        retention_times = retention_times_raa14
        compounds = compounds_raa14
        compound_types = compound_types_raa14
        compound_types_plot = compound_types_plot_raa14
        data_raa = data[data["tape"]=="raa14"]
    if raa_type == 'raa15':
        retention_times = retention_times_raa15
        compounds = compounds_raa15
        compound_types = compound_types_raa15
        compound_types_plot = compound_types_plot_raa15
        data_raa = data[data["tape"]=="raa15"]
    if raa_type == 'raa16':
        retention_times = retention_times_raa16
        compounds = compounds_raa16
        compound_types = compound_types_raa16
        compound_types_plot = compound_types_plot_raa16
        data_raa = data[data["tape"]=="raa16"]
    data_raa.reset_index(inplace=True, drop=True)


    retention_to_compound = dict(zip(retention_times, compounds))
    def retention_to_compound_func(retention_time):
        return retention_to_compound.get(retention_time, 'Unknown')
    compound_to_compound_type = dict(zip(compounds, compound_types))
    def compound_to_compound_type_func(compound):
        return compound_to_compound_type.get(compound, 'Unknown')
    retention_to_compound_type = dict(zip(retention_times, compound_types))
    def retention_to_compound_type_func(retention_time):
        return retention_to_compound_type.get(retention_time, 'Unknown')


    # Calculate peak areas
    columns = ['triplicat', 'file name'] + retention_times
    peak_raa_df = pd.DataFrame(columns=columns)
    for index, row_data in data_raa.iterrows():
        areas = []
        for rt in retention_times:        
            start = find_peak_range(data_raa, index, 1, 35, rt)[0]
            end = find_peak_range(data_raa, index, 1, 35, rt)[1]
            area = peak_area(data_raa, index, start, end)[0]
            areas.append(area)
        new_row = [row_data['triplicat'], row_data['file name']] + areas
        peak_raa_df.loc[len(peak_raa_df)] = new_row
    peak_raa_df.iloc[:, 2:] = peak_raa_df.iloc[:, 2:].clip(lower=0)

    # Calculate relative intensities
    peak_raa_ri_df = peak_raa_df.copy()
    peak_columns = peak_raa_ri_df.columns[2:]
    peak_raa_ri_df['total_intensity'] = peak_raa_df[peak_columns].sum(axis=1)
    for col in peak_columns:
        peak_raa_ri_df[col] = (peak_raa_ri_df[col] / peak_raa_ri_df['total_intensity']) * 100
    peak_raa_ri_df = peak_raa_ri_df.drop(columns=['total_intensity'])

    return (peak_raa_ri_df, retention_times, compounds, compound_types, compound_types_plot, data_raa, retention_to_compound_func, 
            compound_to_compound_type_func, retention_to_compound_type_func, peak_raa_df)

In [ ]:
# Représentation du graph par composé
def plot_bars_compounds():
    width = 0.15  # Width of each bar
    fig, ax = plt.subplots(figsize=(14, 8))

    x = np.arange(len(compounds))
    multiplier = 0

    vertical_lines_positions = []
    compound_type_positions = []

    for i, triplicat in enumerate(peak_raa_ri_df['triplicat'].unique()):
        xi = 0
        compounds_label = []
        data_plot = peak_raa_ri_df[peak_raa_ri_df['triplicat'] == triplicat].dropna()
        triplicat_label_added = False

        for compound_type in compound_types_plot:
            start_position = xi

            for j, retention_time in enumerate(retention_times):
                if retention_to_compound_type_func(retention_time) == compound_type:
                    compounds_label.append(f'{retention_to_compound_func(retention_time)} ; {retention_time}')
                    relative_intensity_plot = data_plot[retention_time][i]
                    standard_deviation_plot = std_devs.iloc[j]
                    
                    # Add label only once per age group
                    if not triplicat_label_added:
                        ax.bar(xi + multiplier * width, relative_intensity_plot, width, label=f'Triplicat: {triplicat}', alpha=0.7)
                        triplicat_label_added = True
                        ax.errorbar(xi + multiplier * width, relative_intensity_plot, yerr=standard_deviation_plot, fmt="+", color="black", alpha=0.7)
                        
                    else:
                        ax.bar(xi + multiplier * width, relative_intensity_plot, width, alpha=0.7, 
                               color=ax.patches[-1].get_facecolor())
                        ax.errorbar(xi + multiplier * width, relative_intensity_plot, yerr=standard_deviation_plot, fmt="+", color="black", alpha=0.7)

                    xi += 1

            end_position = xi

            if i == 0:  # Add vertical lines only once, for the first age group
                vertical_lines_positions.append(end_position - 0.5 * width)
                compound_type_positions.append((start_position + end_position) / 2)

        multiplier += 1

    # Draw vertical lines between compound type areas and add compound type names
    compound_types_ordered = compound_types_plot
    for position, compound_type in zip(vertical_lines_positions, compound_types_ordered):
        ax.axvline(position, color='grey', linestyle='--')
        ax.text(position, ax.get_ylim()[1], compound_type, verticalalignment='top', horizontalalignment='right', 
                rotation=90, fontsize=10, color='grey', alpha=0.7)

    ax.set_xlabel('Compounds')
    ax.set_ylabel('Relative Intensity (%)')
    ax.set_title('Relative Intensities of Peaks for Each Compound')
    ax.set_xticks(x + width * ((len(peak_raa_ri_df['triplicat'].unique()) - 1) / 2))
    ax.set_xticklabels(compounds_label, rotation=90)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.show()

In [ ]:
# Représentation du graph par catégorie
def plot_bars_compound_types():
    width = 0.15  # Width of each bar
    fig, ax = plt.subplots(figsize=(14, 8))

    x = np.arange(len(compound_types_plot))
    multiplier = 0

    vertical_lines_positions = []
    compound_type_positions = []

    for i, triplicat in enumerate(peak_raa_ri_df['triplicat'].unique()):
        xi = 0
        data_plot = peak_raa_ri_df[peak_raa_ri_df['triplicat'] == triplicat].dropna()
        triplicat_label_added = False

        for compound_type in compound_types_plot:
            total_relative_intensity_plot = 0
            standard_deviation_plot = std_devs_by_compound_type.loc[compound_type]

            for j, retention_time in enumerate(retention_times):
                if retention_to_compound_type_func(retention_time) == compound_type:
                    relative_intensity_plot = data_plot[retention_time][i]
                    total_relative_intensity_plot += relative_intensity_plot

            # Add label only once per age group
            if not triplicat_label_added:
                ax.bar(xi + multiplier * width, total_relative_intensity_plot, width, label=f'Triplicat: {triplicat}', alpha=0.7)
                ax.errorbar(xi + multiplier * width, total_relative_intensity_plot, yerr=standard_deviation_plot, fmt="+", color="black", alpha=0.7)
                triplicat_label_added = True
            else:
                ax.bar(xi + multiplier * width, total_relative_intensity_plot, width, alpha=0.7, color=ax.patches[-1].get_facecolor())
                ax.errorbar(xi + multiplier * width, total_relative_intensity_plot, yerr=standard_deviation_plot, fmt="+", color="black", alpha=0.7)

            xi += 1

        multiplier += 1

    ax.set_xlabel('Compound types')
    ax.set_ylabel('Relative Intensity (%)')
    ax.set_title('Relative Intensities of Peaks for Each Compound Type')
    ax.set_xticks(x + width * ((len(peak_raa_ri_df['triplicat'].unique()) - 1) / 2))
    ax.set_xticklabels(compound_types_plot, rotation=90)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.show()

In [ ]:
# Représentation des intensités relatives et des écarts types mesurés
raa_type = 'raa12'

peak_raa_ri_df, retention_times, compounds, compound_types, compound_types_plot, data_raa, retention_to_compound_func, compound_to_compound_type_func, retention_to_compound_type_func, peak_raa_df = initialise_plot_bars_data()

std_devs = peak_raa_ri_df.iloc[:, 2:].std(axis=0)
compound_type_relative_intensities = {compound_type: [] for compound_type in compound_types_plot}
for compound_type in compound_types_plot:
    relative_intensities = []
    for i in [0,1,2]:
        relative_intensity = 0
        for j, retention_time in enumerate(retention_times):
            if retention_to_compound_type_func(retention_time) == compound_type:
                relative_intensity += peak_raa_ri_df.iloc[i, 2+j]
        relative_intensities.append(relative_intensity)
    compound_type_relative_intensities[compound_type] = relative_intensities
std_devs_by_compound_type = {compound_type: np.std(intensities) for compound_type, intensities in compound_type_relative_intensities.items()}
std_devs_by_compound_type = pd.Series(std_devs_by_compound_type, name='Standard Deviation')

plot_bars_compounds()
plot_bars_compound_types()

print(std_devs)
print(std_devs_by_compound_type)

In [ ]:
# Représentation des intensités relatives et des écarts types mesurés
raa_type = 'raa13'

peak_raa_ri_df, retention_times, compounds, compound_types, compound_types_plot, data_raa, retention_to_compound_func, compound_to_compound_type_func, retention_to_compound_type_func, peak_raa_df = initialise_plot_bars_data()

std_devs = peak_raa_ri_df.iloc[:, 2:].std(axis=0)
compound_type_relative_intensities = {compound_type: [] for compound_type in compound_types_plot}
for compound_type in compound_types_plot:
    relative_intensities = []
    for i in [0,1,2]:
        relative_intensity = 0
        for j, retention_time in enumerate(retention_times):
            if retention_to_compound_type_func(retention_time) == compound_type:
                relative_intensity += peak_raa_ri_df.iloc[i, 2+j]
        relative_intensities.append(relative_intensity)
    compound_type_relative_intensities[compound_type] = relative_intensities
std_devs_by_compound_type = {compound_type: np.std(intensities) for compound_type, intensities in compound_type_relative_intensities.items()}
std_devs_by_compound_type = pd.Series(std_devs_by_compound_type, name='Standard Deviation')

plot_bars_compounds()
plot_bars_compound_types()

print(std_devs)
print(std_devs_by_compound_type)

In [ ]:
# Représentation des intensités relatives et des écarts types mesurés
raa_type = 'raa14'

peak_raa_ri_df, retention_times, compounds, compound_types, compound_types_plot, data_raa, retention_to_compound_func, compound_to_compound_type_func, retention_to_compound_type_func, peak_raa_df = initialise_plot_bars_data()

std_devs = peak_raa_ri_df.iloc[:, 2:].std(axis=0)
compound_type_relative_intensities = {compound_type: [] for compound_type in compound_types_plot}
for compound_type in compound_types_plot:
    relative_intensities = []
    for i in [0,1,2]:
        relative_intensity = 0
        for j, retention_time in enumerate(retention_times):
            if retention_to_compound_type_func(retention_time) == compound_type:
                relative_intensity += peak_raa_ri_df.iloc[i, 2+j]
        relative_intensities.append(relative_intensity)
    compound_type_relative_intensities[compound_type] = relative_intensities
std_devs_by_compound_type = {compound_type: np.std(intensities) for compound_type, intensities in compound_type_relative_intensities.items()}
std_devs_by_compound_type = pd.Series(std_devs_by_compound_type, name='Standard Deviation')

plot_bars_compounds()
plot_bars_compound_types()

print(std_devs)
print(std_devs_by_compound_type)

In [ ]:
# Représentation des intensités relatives et des écarts types mesurés
raa_type = 'raa15'

peak_raa_ri_df, retention_times, compounds, compound_types, compound_types_plot, data_raa, retention_to_compound_func, compound_to_compound_type_func, retention_to_compound_type_func, peak_raa_df = initialise_plot_bars_data()

std_devs = peak_raa_ri_df.iloc[:, 2:].std(axis=0)
compound_type_relative_intensities = {compound_type: [] for compound_type in compound_types_plot}
for compound_type in compound_types_plot:
    relative_intensities = []
    for i in [0,1,2]:
        relative_intensity = 0
        for j, retention_time in enumerate(retention_times):
            if retention_to_compound_type_func(retention_time) == compound_type:
                relative_intensity += peak_raa_ri_df.iloc[i, 2+j]
        relative_intensities.append(relative_intensity)
    compound_type_relative_intensities[compound_type] = relative_intensities
std_devs_by_compound_type = {compound_type: np.std(intensities) for compound_type, intensities in compound_type_relative_intensities.items()}
std_devs_by_compound_type = pd.Series(std_devs_by_compound_type, name='Standard Deviation')

plot_bars_compounds()
plot_bars_compound_types()

print(std_devs)
print(std_devs_by_compound_type)

In [ ]:
# Représentation des intensités relatives et des écarts types mesurés
raa_type = 'raa16'

peak_raa_ri_df, retention_times, compounds, compound_types, compound_types_plot, data_raa, retention_to_compound_func, compound_to_compound_type_func, retention_to_compound_type_func, peak_raa_df = initialise_plot_bars_data()

std_devs = peak_raa_ri_df.iloc[:, 2:].std(axis=0)
compound_type_relative_intensities = {compound_type: [] for compound_type in compound_types_plot}
for compound_type in compound_types_plot:
    relative_intensities = []
    for i in [0,1,2]:
        relative_intensity = 0
        for j, retention_time in enumerate(retention_times):
            if retention_to_compound_type_func(retention_time) == compound_type:
                relative_intensity += peak_raa_ri_df.iloc[i, 2+j]
        relative_intensities.append(relative_intensity)
    compound_type_relative_intensities[compound_type] = relative_intensities
std_devs_by_compound_type = {compound_type: np.std(intensities) for compound_type, intensities in compound_type_relative_intensities.items()}
std_devs_by_compound_type = pd.Series(std_devs_by_compound_type, name='Standard Deviation')

plot_bars_compounds()
plot_bars_compound_types()

print(std_devs)
print(std_devs_by_compound_type)